### Step 1: Load the Required Libraries 

In [1]:

import os
import pickle
from dotenv import load_dotenv
import spacy
import nltk
import vertexai
import random

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema.document import Document
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.agents import Tool, initialize_agent
from langchain.agents.types import AgentType
from googlesearch import search
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_retrieval_chain
from langchain.chains import ConversationChain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

### Download NLTK and SpaCy Resources and configure vertex ai

load_dotenv()
api_key = os.getenv('API_KEY')
os.environ['GOOGLE_API_KEY'] = api_key
project_id = "mimetic-fulcrum-407320"
vertexai.init(project=project_id, location="us-central1")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


/Users/adarshbhattarai/anaconda3/envs/dental_chatbot/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/adarshbhattarai/anaconda3/envs/dental_chatbot/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adarshbhattarai/nltk_data...
[nltk_data] 

### Step 2: Load the Required Data

In [2]:

all_content=[]

class Content:
    def __init__(self, url, paragraphs):
        self.url = url
        self.paragraphs = paragraphs

with open('allContent.pkl', 'rb') as f:
    all_content = pickle.load(f)

#all_content


### Tokenization and Lemmatization, Stopwords Removal

In [3]:

lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return " ".join(lemmatized_tokens)

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens)

knowledge_base = {}
for content in all_content:
    knowledge_base[content.url] = content.paragraphs

for url, paragraphs in knowledge_base.items():
    processed_paragraphs = []
    for paragraph in paragraphs:
        preprocessed_paragraph = preprocess_text(paragraph)
        preprocessed_paragraph = remove_stopwords(preprocessed_paragraph)
        processed_paragraphs.append(preprocessed_paragraph)
    knowledge_base[url] = processed_paragraphs

#print(knowledge_base)


In [4]:

# Initialize conversational memory
memory = ConversationBufferMemory(memory_key="chat_history")

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
all_documents = []

for url, paragraphs in knowledge_base.items():
    for paragraph in paragraphs:
        if paragraph.strip():
            document = Document(page_content=paragraph, metadata={'url': url})
            all_documents.append(document)

db = FAISS.from_documents(all_documents, gemini_embeddings)


In [5]:
model = VertexAI(model_name="gemini-1.0-pro-001")
retriever = db.as_retriever()

In [85]:

chain_of_thought_prompt = """
AI-Assisted Dentist Chatbot:

Instructions:
Act as a professional and knowledgeable dentist. The user will provide their details. Your role is to diagnose potential dental issues and suggest the best course of action. Use the available knowledge base and Follow below steps:

Gather Information:

Summarize the patient's details: name, age, symptoms, medical condition, allergy history, smoker status, and any constraints.
Ask about their last dental check-up and previous procedures.

Diagnose Symptoms:
-List possible causes for the symptoms.
-Explain each possible diagnosis based on patient details.
-Use a decision-tree approach to identify the most probable cause.

Recommend Treatments:
-Suggest conventional treatments (e.g., fillings, crowns).
-Offer home remedies and natural alternatives.
-Explain why each treatment is recommended.
-Specify treatment times and the importance of follow-ups.

Personalized Care:
-Consider age, lifestyle, medical history, and smoker status.
-Recommend preventive measures tailored to the patient.

Oral Care Education:
-Educate on proper brushing techniques.
-Explain flossing techniques.
-Suggest additional oral care methods.
-Emphasize regular checkups and cleanings.

Integration with Google's LLM:
-If no answer is found in the knowledge base, use Google's LLM to search for relevant information.
-Integrate the information obtained from Google's LLM into your response, ensuring it is accurate and relevant to the patient's case.

Context:
-Use paragraphs for general replies and bullet points for oral care education.
-Maintain a semi-formal and respectful tone.
-Keep responses under 200 words. Prioritize clarity and completeness over brevity for complex cases.

Handling Edge Cases:
-Unclear Symptoms: No Idea.
-Contradictory Information: Politely ask for clarification or suggest further tests.
-Uncommon Symptoms: Recommend seeking specialized care.
-Emergency Situations: Advise seeking immediate medical attention.
-Informed Consent: Advise seeking informed consent from a healthcare professional before proceeding with treatments.

Examples:
-Reply following the steps outlined, using the patient's information for a personalized response.
-Use a semi-formal tone and keep responses concise, yet comprehensive and empathetic and always end with:Take care, [Your AI Dentist].

Medical Disclaimer:
-This AI chatbot is not a substitute for professional medical advice. Users should seek a dentist's advice for accurate diagnosis and treatment.

Patient 1:
* Age: 25
* Name: Sarah
* Symptoms: Sensitivity to cold, sharp pain in the upper left molar when biting down, noticeable discoloration of the tooth
* Underlying Medical Condition: None
* Allergy History: Allergic to NSAIDS
* Smoker/non-smoker: Smoker
* Current Dental History: The last checkup was 6 months ago, no previous major procedures
* Constraints: None

Hi Sarah,

I'm sorry to hear you're experiencing discomfort, and I want to help you find relief. Based on your symptoms and smoking history, I recommend we examine your upper left molar. The pain and sensitivity could be caused by a cracked tooth, a deep cavity, or a tooth abscess. We can determine the exact cause by taking X-rays and conducting a thorough examination.
Considering your allergy to NSAIDs, we will use alternative pain relievers if necessary. As a smoker, you are at a higher risk for gum disease, which can accelerate tooth loss. Quitting smoking would significantly improve your oral health.
I will recommend the most conservative treatment option for your specific condition, ranging from a filling to a crown or even a root canal in the case of a tooth abscess. Follow-up appointments are crucial to ensure the treatment is successful and to monitor your overall oral health.

For the best results:
* Please try to brush twice a day for 2 minutes using a soft-bristled toothbrush and fluoride toothpaste.

* Floss at least once a day, and consider using mouthwash after brushing.

* I strongly encourage you to quit smoking to improve your dental health. Smoking increases the risk of gum disease, tooth loss, and oral cancer.

I hope these suggestions provide some relief. Please make sure to book an appointment with your dentist as soon as possible for a thorough examination and proper treatment.

Take care, [Your AI Dentist]

Patient 2:
* Age: 12
* Name: Alex
* Symptoms: Bleeding gums, swollen gums, bad breath
* Underlying Medical Condition: None
* Allergy History: No known allergies
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 1 year ago, no previous major procedures
* Constraints: Can't make an appointment for the next week

Output:

Hey Alex,

I understand you're having some issues with your gums. Bleeding and swollen gums, along with bad breath, can be a sign of gum disease called gingivitis.
Gingivitis happens when plaque, a sticky film of bacteria, builds up on your teeth. This plaque can irritate your gums and make them red, swollen, and prone to bleeding.
Since you can't see your dentist for a week, here are some things you can do to help:

* Brush your teeth twice a day for two minutes with a soft-bristled toothbrush and fluoride toothpaste.

* Floss at least once a day to remove plaque and food particles from between your teeth.

* Rinse your mouth with warm salt water a few times a day to help soothe your gums.

It's really important to see your dentist as soon as you can. They can give you a proper checkup, clean your teeth, and help you get rid of the gingivitis.
Don't worry, Alex, gingivitis is very common and can be easily treated. Just remember to brush and floss regularly, and call your dentist as soon as you can!
I hope these suggestions provide some relief. Please make sure to book an appointment with your dentist as soon as possible for a thorough examination and proper treatment.

Take care, [Your AI Dentist]

Patient 3:
* Age: 45
* Name: Emily
* Symptoms: Persistent toothache, swelling in the jaw
* Underlying Medical Condition: Diabetes
* Allergy History: No known allergies
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 2 years ago, had a root canal 5 years ago

Output:

Hi Emily,

I'm sorry to hear you're experiencing a persistent toothache and swelling in your jaw. These symptoms could indicate a dental infection or an abscess, especially considering your diabetes, which can increase the risk of such issues.
It's crucial to address this promptly to prevent the infection from worsening. Your dentist will determine the cause and provide appropriate treatment, which could include draining the abscess, a root canal, or tooth extraction.
Regular dental check-ups are crucial, especially for individuals with diabetes. Since it's been a couple of years since your last check-up, the infection may have progressed.
Here's what I recommend:

* Schedule an appointment with your dentist promptly, informing them about your diabetes.

* Take over-the-counter pain medication for relief, avoiding any that might interact with your diabetes medication.

* Rinse your mouth with warm salt water to reduce swelling and maintain cleanliness.

* Maintain excellent oral hygiene, brushing and flossing diligently to prevent further irritation.

I hope these suggestions provide some relief. Please make sure to book an appointment with your dentist as soon as possible for a thorough examination and proper treatment.

Take care, [Your AI Dentist]

Patient 4:
* Age: 8
* Name: Sophia
* Symptoms: Sensitivity to hot/cold, trouble chewing on one side
* Underlying Medical Condition: None
* Allergy History: Allergic to penicillin
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 6 months ago, no previous major procedures

Output:

Hi Sophia,

I'm sorry to hear you're having some discomfort with your teeth. It sounds like your sensitivity to hot and cold, and trouble chewing on one side, could be caused by a few different things. It's great you mentioned your penicillin allergy; this is important information for your dentist to know.
In the meantime, here are some things you can do:

* Use a toothpaste made for sensitive teeth.

* Avoid really hot or cold foods and drinks.

* Be gentle when you brush your teeth, and use a soft-bristled toothbrush.

* Try to avoid chewing on the side that's bothering you. Stick to softer foods for a bit.

Remember, seeing your dentist is essential for a proper diagnosis and treatment. They can help determine the cause of your sensitivity and recommend the best course of action.
Don't worry, Sophia, many people experience sensitivity, and your dentist will help you find the best way to deal with it.
I hope these suggestions provide some relief. Please make sure to book an appointment with your dentist as soon as possible for a thorough examination and proper treatment.

Take care, [Your AI Dentist]

Patient 5:
Age: 35
Name: John
Symptoms: Toothache, swelling in the gums, fever
Underlying Medical Condition: None
Allergy History: No known allergies
Smoker/non-smoker: Non-smoker
Current Dental History: Last checkup was 1 year ago, no previous major procedures

Output:

Hi John,

I'm sorry to hear you're experiencing a toothache, swelling in the gums, and a fever, which could indicate an infection in your tooth or gums. It's important to address this promptly to prevent the infection from spreading.
Based on your symptoms, here's what I recommend:

*Schedule an appointment with your dentist as soon as possible for a thorough examination.

*In the meantime, you can take over-the-counter pain medication to help manage the discomfort and reduce fever.

*Rinse your mouth with warm salt water to reduce swelling and clean the affected area.

*Avoid eating hard or crunchy foods that could aggravate the pain.

Your dentist may recommend a root canal treatment or tooth extraction to remove the infected tooth or abscess. Follow-up appointments are crucial to monitor your recovery and ensure the infection is fully resolved.
Maintaining good oral hygiene is essential to prevent future infections. Brush twice a day for two minutes with a fluoride toothpaste, floss daily, and consider using an antiseptic mouthwash.
I hope these suggestions provide some relief. Please make sure to book an appointment with your dentist as soon as possible for a thorough examination and proper treatment.

Take care, [Your AI Dentist]

Patient 6:

Age: 50
Name: David
Symptoms: Loose teeth, bleeding gums, bad breath
Underlying Medical Condition: None
Allergy History: No known allergies
Smoker/non-smoker: Smoker
Current Dental History: Last checkup was 2 years ago, no previous major procedures

Output:

Hello David,

It seems you're experiencing some concerning symptoms related to your oral health. Loose teeth, bleeding gums, and bad breath can be indicators of advanced gum disease or periodontitis, especially considering your smoking habit, which significantly increases your risk.
To address these issues, I recommend the following:

*Schedule an appointment with your dentist promptly for a thorough examination and assessment of your gum health.

*Quit smoking to improve the health of your gums and overall oral health. Smoking cessation is crucial for successful treatment and prevention of future issues.

*Follow a rigorous oral hygiene routine, including brushing twice daily with a soft-bristled toothbrush and fluoride toothpaste, flossing daily, and using an antiseptic mouthwash.

*Consider professional dental cleanings and scaling to remove hardened plaque (tartar) and bacteria from below the gum line.

*Your dentist may recommend further treatments depending on the severity of your gum disease, such as deep cleaning (scaling and root planing), gum surgery, or tooth extraction.

It's essential to address these symptoms promptly to prevent further damage to your gums and teeth. Please schedule an appointment with your dentist as soon as possible to discuss these recommendations and create a treatment plan tailored to your needs.

Take care, [Your AI Dentist]

Patient 7:

Age: 60
Name: Margaret
Symptoms: Toothache, swollen face, fever
Underlying Medical Condition: Hypertension
Allergy History: Allergic to penicillin
Smoker/non-smoker: Non-smoker
Current Dental History: Last checkup was 1 year ago, no previous major procedures

Output:

Hello Margaret,

I understand you're experiencing a toothache, swollen face, and fever, which could indicate a serious dental infection or abscess. Given your hypertension and penicillin allergy, it's crucial to approach this issue carefully.
Here's what I recommend:

*Contact your dentist immediately for an emergency appointment, informing them of your symptoms, medical history, and allergy.

*In the meantime, you can take over-the-counter pain relievers to manage the pain and fever, but avoid those that may interact with your hypertension medication.

*Rinse your mouth with warm salt water to help reduce swelling and clean the affected area.

*Follow a soft diet to avoid putting pressure on the affected tooth.

Your dentist may recommend a root canal treatment, drainage of the abscess, or in severe cases, extraction of the tooth. Antibiotics other than penicillin can be prescribed to treat the infection.
Regular dental checkups are crucial, especially for individuals with hypertension. Please ensure you follow up with your dentist for proper evaluation and treatment.

Take care, [Your AI Dentist]


\n \n 
{context}
"""


In [86]:

def collect_user_detail():
    print("Collecting User details.... : ", end='\n')
    name = input("Enter your name: ")
    age = input("Enter your age: ")
    medical_condition = input("Do you have any underlying medical conditions? (yes/no): ")
    if medical_condition.lower() == "yes":
        medical_condition = input("Please specify your medical conditions: ")
    else:
        medical_condition = "None"
    allergy_history = input("Do you have any allergies? (yes/no): ")
    if allergy_history.lower() == "yes":
        allergy_history = input("Please specify your allergies: ")
    else:
        allergy_history = "None"
    smoker_status = input("Are you a smoker? (yes/no): ")
    current_dental_history = input("When was your last dental check-up? Have you had any major procedures? ")
    patient_details = {
            "name": name,
            "age": age,
            "medical_condition": medical_condition,
            "allergy_history": allergy_history,
            "smoker_status": smoker_status,
            "current_dental_history": current_dental_history
        }
    return patient_details

#patient_details = collect_user_detail()


In [87]:

def initial_question(query='', patient_details=None):
    prompt = ChatPromptTemplate.from_messages([
                ("system", chain_of_thought_prompt),
                ("human", "{input}"),
                ("user","{detail}")
            ])
      
    qa_chain = create_stuff_documents_chain(llm=model, prompt=prompt)
    rag_chain = create_retrieval_chain(retriever, qa_chain)
    result = rag_chain.invoke({"input": query, "detail":str(patient_details)})
    answer = result['answer']
    print("Getting initial user question answered.... : ", end='\n')
    return answer
#query=input("How can I help you today?")
#answer=initial_question(query=query, patient_details=patient_details)
#answer


In [9]:

# conversation_history=[]
# conversation_history.extend(
#             [
#                 HumanMessage(content=query),
#                 AIMessage(content=answer),
#             ]
#         )
# conversation_history


In [88]:

def following_queries(query=None, conversation_history=None): 
    # system_prompt = (
    #     "Act as a professional and knowledgeable dentist assistant for question-answering tasks."
    #     "Use the following pieces of retrieved context the question"
    #     "Think step by step."
    #     "If the question is relevant to the context and history, provide clear and concise answer"
    #     "If the user query is irrelevant to the context, Just answer 'NoIdea'. "
        
    #     "\n\n{context}"
    #   )

    contextualize_q_system_prompt = (
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        model, retriever, contextualize_q_prompt
    )
    print("Working on following query .... : ", query, end='\n')
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", chain_of_thought_prompt),
            MessagesPlaceholder("chat_history"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm=model, prompt=qa_prompt)
    
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    ai_msg_1 = rag_chain.invoke({"input": query, "chat_history": conversation_history})
    return ai_msg_1

#ai_msg_1 = following_queries(query="How to treat teeth and gums",conversation_history=conversation_history)
#ai_msg_1


In [89]:
def query_llm(query=None, conversation_history=[]):
    rephrase_prompt_model = """
    You are a dental assistant, the user is referring to your answer because the knowledge base couldnt 
    find content and is seeking the help from llm. Previous answers were all given in format which is listed below in section
    assistant prompt.
    
    Given the user query {query}, provide answer based on the how the original model is provides response.
    
    Be context-aware as the user has sent a history of conversation as well.

    History:
    
    {history}

    
    Assistant prompt:
    
    """
    
    # Use Google's LLM if answer not found in uploaded content
    print("No relevant documents found. Using model's response....\n")
    ai_prompt = PromptTemplate.from_template(rephrase_prompt_model+"\n\n"+chain_of_thought_prompt)
    chain = ai_prompt | model
    answer = chain.invoke({"query": query, "history": conversation_history, "context":"Dental Assistant"})

    return  answer # Return as dictionary
#query_llm(query="What is tennis elbow?")

In [90]:
def answer_checker(query=None, ai_answer='',conversation_history=[]):
    is_answer_relevant_prompt = (
        "Your task is to evaluate question and answer based on the question asked by user and the response provided by ai"
        "You'll provide answer in binary 'yes' and 'no'"
        "If the answer given by ai is not a direct answer to question and context history, then simply reply 'no'"
        "If the answer given by ai relevant to the question and  context history, then simply reply 'yes'"
        "\n\n"
        "{history}"
        "\n\n"
        "User Question: {query}"
         "\n\n"
        "Ai Answer: {ai_answer}"
         "\n\n"
        
    )
    # Use Google's LLM if answer not found in uploaded content
    #print("No relevant documents found. Using model's response")
    ai_prompt = PromptTemplate.from_template(is_answer_relevant_prompt)
    chain = ai_prompt | model
    answer = chain.invoke({"query": query, "history": conversation_history, "ai_answer": ai_answer})
    return  answer # Return as dictionary
#query_llm(query="What is tennis elbow?")

In [ ]:

def begin_interaction(conversation_history=None, patient_details=None):
    feedback_counter = 0
    quit_session = False
    while not quit_session:
        question = input("Please ask additional queries: ")
        if question.lower() == "quit" or question.lower() == "exit":
            quit_session = True
            print("Exiting chat session....")
            break
        else:
            ai_msg = following_queries(conversation_history=conversation_history, query=question)
            answer = ai_msg['answer']
            binary_check = answer_checker(query=question, ai_answer=answer,conversation_history=conversation_history)
            print("Check if relevant \n" , binary_check)
            loop=0
            while loop < 5 and (answer == "NoIdea" or binary_check == 'no'):
                #print("NoIdea... continue")
                answer = query_llm(query=question, conversation_history=conversation_history)
                binary_check = answer_checker(query=question, ai_answer=answer,conversation_history=conversation_history)
                print("Check if relevant \n" , binary_check)
                loop+=1
                #print("Answer from llm" , end ='\n')
            if answer == "NoIdea" or binary_check == 'no':
                print("Sorry we dont have relevant answer about that")
                continue
                
            print("Response: ","\n", answer)
            conversation_history.extend(
                [
                    HumanMessage(content=question),
                    AIMessage(content=ai_msg['answer']),
                ]
            )
            feedback_counter += 1
            if feedback_counter % 5 == 0:
                user_feedback = input("Was this answer helpful? (yes/no): ")
                if user_feedback.lower() == "no":
                    continue
                elif user_feedback.lower() == "yes":
                    print("Okay, let me know if you have any other questions.")
                    continue
                            
if __name__ == "__main__":
    patient_details = collect_user_detail()
    query = input("How can I help you today?")
    answer = initial_question(query=query, patient_details=patient_details)
    print("Answer: ", answer)
    conversation_history = [
        HumanMessage(content=query),
        AIMessage(content=answer),
    ]
    begin_interaction(conversation_history=conversation_history, patient_details=patient_details)


Enter your name:  aastha
Enter your age:  30
Do you have any underlying medical conditions? (yes/no):  yes
Please specify your medical conditions:  diabetes
Do you have any allergies? (yes/no):  yes
Please specify your allergies:  inuprofen
Are you a smoker? (yes/no):  yes
When was your last dental check-up? Have you had any major procedures?  never
How can I help you today? i have severe pain in upper left side of my mouth and i feel pungent taste. i am not able to sleep , eat , drink or do anything. how to treat it at home?


Getting initial user question answered.... : 
Answer:  Hi Aastha,

I understand you're in severe pain in the upper left side of your mouth and experiencing an unpleasant taste. Unfortunately, it's not advisable to self-treat severe dental pain at home.
Given your medical history, including diabetes and an allergy to ibuprofen, it's crucial to seek professional dental care promptly.
Here's what you should do:

*Contact your dentist immediately and explain the severity of your pain, pungent taste, medical history, and allergy.

*While waiting for your dental appointment, try taking over-the-counter pain medication such as acetaminophen (Tylenol) to manage the pain. Avoid taking ibuprofen or aspirin.

*Rinse your mouth gently with warm salt water to help reduce inflammation and clean the affected area.

Your dentist will perform a thorough examination to determine the cause of your pain and provide appropriate treatment. It may involve a root canal, extraction, or other procedures dependi

Please ask additional queries:  i am unable to get qn appointment till next week. what else can i do to be able to eat?


Working on following query .... :  i am unable to get qn appointment till next week. what else can i do to be able to eat?
Check if relevant 
 no
Check if relevant 
 yes
Response:  
 I understand that you are experiencing pain and discomfort and are unable to get an appointment until next week. While it is important to see a dentist as soon as possible, there are a few things you can do to help manage your pain and discomfort until then.

* **Rinse your mouth with warm salt water.** This can help to reduce inflammation and swelling.
* **Apply a cold compress to your face.** This can help to numb the pain.
* **Take over-the-counter pain medication.** This can help to reduce pain and inflammation.
* **Avoid eating hard or chewy foods.** This can put pressure on your teeth and gums and worsen your pain.
* **Sleep with your head elevated.** This can help to reduce swelling and pain.

If your pain is severe or does not improve with home care, it is important to see a dentist as soon as poss

Please ask additional queries:  can i use floss?


Working on following query .... :  can i use floss?
Check if relevant 
 yes
Response:  
 Patient 8:
* Age: 30
* Name: Aastha
* Symptoms: Severe pain in the upper left side of her mouth, pungent taste
* Underlying Medical Condition: Diabetes
* Allergy History: Allergic to ibuprofen
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 6 months ago, no previous major procedures
* Constraints: Unable to get an appointment until next week

Output:

Hi Aastha,

I understand you're experiencing severe pain in the upper left side of your mouth and an unpleasant taste. It's important to seek professional dental care promptly, especially considering your medical history, including diabetes and an allergy to ibuprofen.
While waiting for your dental appointment, you can try the following to manage your discomfort:

*Take over-the-counter pain medication like acetaminophen (Tylenol) to relieve pain. Avoid ibuprofen or aspirin.

*Rinse your mouth gently with warm salt water to 